This notebook is to load each model, and export the predicted labels for each image in the test sets. This will allow for metric calculation (specificity at 98% sensitivity, reworked precision/recall/f1 etc), but more importantly, for sub-group reporting. Specifically, for example, performance on gender, age, machine etc.

In [3]:
!pip install --quiet opencv-python torch torchvision captum 

In [ ]:
import sys
sys.path.insert(1, '../../')
import helpers, os
from PIL import Image
import cv2, numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from helpers.attribution import convert_to_uint8

device = helpers.models.get_device()

### Load models into CPU

In [5]:

models_base_pth = "/home/local/data/sophie/padcxr14_models/step_decay_cxr14/arch_seg/"
full_config_path = "full_network/{}/lr_step_decay_10_epochs_0.1_rate/model_lr_0.0005_bsz_16_mom_0.99_seed_{}_pos-weight_1.0.pth/model_17.pth"
training_type = "full"
# load base models to cpu
base_full_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("base","42")),device="cpu")
base_full_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("base","43")),device="cpu")
base_full_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("base","44")),device="cpu")

# load grey models to cpu
grey_full_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("grey","42")),device="cpu")
grey_full_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("grey","43")),device="cpu")
grey_full_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("grey","44")),device="cpu")

# load single models to cpu
single_full_model_1 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,full_config_path.format("single","42")),
    device="cpu",
    single=True
)
single_full_model_2 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,full_config_path.format("single","43")),
    device="cpu",
    single=True
)
single_full_model_3 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,full_config_path.format("single","44")),
    device="cpu",
    single=True
)

# load base models to cpu
base65_full_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("base65","42")),device="cpu")
base65_full_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("base65","43")),device="cpu")
base65_full_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("base65","44")),device="cpu")

# load grey models to cpu
grey65_full_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("grey65","42")),device="cpu")
grey65_full_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("grey65","43")),device="cpu")
grey65_full_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,full_config_path.format("grey65","44")),device="cpu")

# load single models to cpu
single65_full_model_1 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,full_config_path.format("single65","42")),
    device="cpu",
    single=True
)
single65_full_model_2 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,full_config_path.format("single65","43")),
    device="cpu",
    single=True
)
single65_full_model_3 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,full_config_path.format("single65","44")),
    device="cpu",
    single=True
)
# ==================================================
first_config_path = "first_layer_freeze/{}/lr_step_decay_10_epochs_0.1_rate/model_lr_0.0005_bsz_16_mom_0.99_seed_{}_pos-weight_1.0.pth/model_17.pth"
# load base models to cpu
base_first_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("base","42")),device="cpu")
base_first_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("base","43")),device="cpu")
base_first_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("base","44")),device="cpu")

# load grey models to cpu
grey_first_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("grey","42")),device="cpu")
grey_first_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("grey","43")),device="cpu")
grey_first_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("grey","44")),device="cpu")

# load single models to cpu
single_first_model_1 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,first_config_path.format("single","42")),
    device="cpu",
    single=True
)
single_first_model_2 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,first_config_path.format("single","43")),
    device="cpu",
    single=True
)
single_first_model_3 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,first_config_path.format("single","44")),
    device="cpu",
    single=True
)

# load base models to cpu
base65_first_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("base65","42")),device="cpu")
base65_first_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("base65","43")),device="cpu")
base65_first_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("base65","44")),device="cpu")

# load grey models to cpu
grey65_first_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("grey65","42")),device="cpu")
grey65_first_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("grey65","43")),device="cpu")
grey65_first_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,first_config_path.format("grey65","44")),device="cpu")

# load single models to cpu
single65_first_model_1 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,first_config_path.format("single65","42")),
    device="cpu",
    single=True
)
single65_first_model_2 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,first_config_path.format("single65","43")),
    device="cpu",
    single=True
)
single65_first_model_3 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,first_config_path.format("single65","44")),
    device="cpu",
    single=True
)

# ==================================================

half_config_path = "half_network/{}/lr_step_decay_10_epochs_0.1_rate/model_lr_0.01_bsz_16_mom_0.9_seed_{}_pos-weight_1.0.pth/model_17.pth"
# load base models to cpu
base_half_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("base","42")),device="cpu")
base_half_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("base","43")),device="cpu")
base_half_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("base","44")),device="cpu")

# load grey models to cpu
grey_half_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("grey","42")),device="cpu")
grey_half_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("grey","43")),device="cpu")
grey_half_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("grey","44")),device="cpu")

# load single models to cpu
single_half_model_1 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,half_config_path.format("single","42")),
    device="cpu",
    single=True
)
single_half_model_2 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,half_config_path.format("single","43")),
    device="cpu",
    single=True
)
single_half_model_3 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,half_config_path.format("single","44")),
    device="cpu",
    single=True
)

# load base models to cpu
base65_half_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("base65","42")),device="cpu")
base65_half_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("base65","43")),device="cpu")
base65_half_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("base65","44")),device="cpu")

# load grey models to cpu
grey65_half_model_1 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("grey65","42")),device="cpu")
grey65_half_model_2 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("grey65","43")),device="cpu")
grey65_half_model_3 = helpers.models.load_trained_resnet50(os.path.join(models_base_pth,half_config_path.format("grey65","44")),device="cpu")

# load single models to cpu
single65_half_model_1 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,half_config_path.format("single65","42")),
    device="cpu",
    single=True
)
single65_half_model_2 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,half_config_path.format("single65","43")),
    device="cpu",
    single=True
)
single65_half_model_3 = helpers.models.load_trained_resnet50(
    os.path.join(models_base_pth,half_config_path.format("single65","44")),
    device="cpu",
    single=True
)

# ==================================================


### Load Datasets

In [6]:
data_root = "/home/local/data/sophie/node21_combined"
dataset_name = "cxr14"
process = "arch"
std_dir = "std_1024"
crop_size = 512
bsz = 1

bbox_df = helpers.bboxes.read_bounding_boxes("/home/local/data/sophie/node21_combined/all_bboxes.csv")
cxr_test = helpers.dataloading.load_dataset_with_paths(
    dataset_path=os.path.join(data_root, dataset_name, "test"),
    dataset_name=dataset_name,
    process=process,
    crop_size=crop_size,
    batch_size=bsz,
    shuffle=False
)

single_cxr_test = helpers.dataloading.load_dataset_with_paths(
    dataset_path=os.path.join(data_root, dataset_name, "test"),
    dataset_name=dataset_name,
    process=process,
    crop_size=crop_size,
    batch_size=bsz,
    shuffle=False,
    single_channel=True
)
#====================================================
dataset_name = "padchest"
pad_test = helpers.dataloading.load_dataset_with_paths(
    dataset_path=os.path.join(data_root, dataset_name, "test"),
    dataset_name=dataset_name,
    process=process,
    crop_size=crop_size,
    batch_size=bsz,
    shuffle=False
)

single_pad_test = helpers.dataloading.load_dataset_with_paths(
    dataset_path=os.path.join(data_root, dataset_name, "test"),
    dataset_name=dataset_name,
    process=process,
    crop_size=crop_size,
    batch_size=bsz,
    shuffle=False,
    single_channel=True
)
#====================================================
dataset_name = "jsrt"
jsrt_test = helpers.dataloading.load_dataset_with_paths(
    dataset_path=os.path.join(data_root, dataset_name, "test"),
    dataset_name=dataset_name,
    process=process,
    crop_size=crop_size,
    batch_size=bsz,
    shuffle=False
)

single_jsrt_test = helpers.dataloading.load_dataset_with_paths(
    dataset_path=os.path.join(data_root, dataset_name, "test"),
    dataset_name=dataset_name,
    process=process,
    crop_size=crop_size,
    batch_size=bsz,
    shuffle=False,
    single_channel=True
)
#====================================================
dataset_name = "openi"
openi_test = helpers.dataloading.load_dataset_with_paths(
    dataset_path=os.path.join(data_root, dataset_name, "test"),
    dataset_name=dataset_name,
    process=process,
    crop_size=crop_size,
    batch_size=bsz,
    shuffle=False
)

single_openi_test = helpers.dataloading.load_dataset_with_paths(
    dataset_path=os.path.join(data_root, dataset_name, "test"),
    dataset_name=dataset_name,
    process=process,
    crop_size=crop_size,
    batch_size=bsz,
    shuffle=False,
    single_channel=True
)

## Evaluate Models

In [35]:
from IPython.display import clear_output
output_base_pth = "/home/local/data/sophie/DGX_GridSearch/raw_predictions/"

### Full Network

#### Base Full Eval

In [22]:
backbone = "base"
training_strategy = "full"
model_list = [base_full_model_1, base_full_model_2, base_full_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [31]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Base65 Full Eval

In [36]:
backbone = "base65"
training_strategy = "full"
model_list = [base65_full_model_1, base65_full_model_2, base65_full_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [37]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Grey Full Eval

In [38]:
backbone = "grey"
training_strategy = "full"
model_list = [grey_full_model_1, grey_full_model_2, grey_full_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [39]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Grey65 Full Eval

In [40]:
backbone = "grey65"
training_strategy = "full"
model_list = [grey65_full_model_1, grey65_full_model_2, grey65_full_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [41]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Single Full Eval

In [42]:
backbone = "single"
training_strategy = "full"
model_list = [single_full_model_1, single_full_model_2, single_full_model_3]
run_seed = [42,43,44]
dataset_list = [single_cxr_test, single_pad_test, single_jsrt_test, single_openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [43]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Single65 Full Eval

In [56]:
backbone = "single65"
training_strategy = "full"
model_list = [single65_full_model_1, single65_full_model_2, single65_full_model_3]
run_seed = [42,43,44]
dataset_list = [single_cxr_test, single_pad_test, single_jsrt_test, single_openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [57]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


### First Layer Network

#### Base First Eval

In [44]:
backbone = "base"
training_strategy = "first"
model_list = [base_first_model_1, base_first_model_2, base_first_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [45]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Base65 First Eval

In [46]:
backbone = "base65"
training_strategy = "first"
model_list = [base65_first_model_1, base65_first_model_2, base65_first_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [47]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Grey First Eval

In [48]:
backbone = "grey"
training_strategy = "first"
model_list = [grey_first_model_1, grey_first_model_2, grey_first_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [49]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Grey65 First Eval

In [50]:
backbone = "grey65"
training_strategy = "first"
model_list = [grey65_first_model_1, grey65_first_model_2, grey65_first_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [51]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Single First Eval

In [52]:
backbone = "single"
training_strategy = "first"
model_list = [single_first_model_1, single_first_model_2, single_first_model_3]
run_seed = [42,43,44]
dataset_list = [single_cxr_test, single_pad_test, single_jsrt_test, single_openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [53]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Single65 First Eval

In [54]:
backbone = "single65"
training_strategy = "first"
model_list = [single65_first_model_1, single65_first_model_2, single65_first_model_3]
run_seed = [42,43,44]
dataset_list = [single_cxr_test, single_pad_test, single_jsrt_test, single_openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [55]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


### Half Network

#### Base Half Eval

In [58]:
backbone = "base"
training_strategy = "half"
model_list = [base_half_model_1, base_half_model_2, base_half_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [59]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n1128


#### Base65 Half Eval

In [60]:
backbone = "base65"
training_strategy = "half"
model_list = [base65_half_model_1, base65_half_model_2, base65_half_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [ ]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

model: 2  dataset 3/4  img: n0252


#### Grey First Eval

In [ ]:
backbone = "grey"
training_strategy = "half"
model_list = [grey_half_model_1, grey_half_model_2, grey_half_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [ ]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

#### Grey65 First Eval

In [ ]:
backbone = "grey65"
training_strategy = "half"
model_list = [grey65_half_model_1, grey65_half_model_2, grey65_half_model_3]
run_seed = [42,43,44]
dataset_list = [cxr_test, pad_test, jsrt_test, openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [ ]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

#### Single First Eval

In [ ]:
backbone = "single"
training_strategy = "half"
model_list = [single_half_model_1, single_half_model_2, single_half_model_3]
run_seed = [42,43,44]
dataset_list = [single_cxr_test, single_pad_test, single_jsrt_test, single_openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [ ]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)

#### Single65 Half Eval

In [ ]:
backbone = "single65"
training_strategy = "half"
model_list = [single65_half_model_1, single65_half_model_2, single65_half_model_3]
run_seed = [42,43,44]
dataset_list = [single_cxr_test, single_pad_test, single_jsrt_test, single_openi_test]
dataset_names = ["cxr14", "padchest", "jsrt", "openi"]

In [ ]:
for i in range(0,len(model_list)):
    # load model to GPU
    model = model_list[i].to(device)
    for j in range(0, len(dataset_list)):
        model_ds_img_names = []
        model_ds_img_labels = []
        model_ds_preds_0 = []
        model_ds_preds_1 = []
        # index tracker for printing
        for curr_data in dataset_list[j]:
            clear_output(wait=True)
            inputs, labels = curr_data[0],curr_data[1]
            paths = np.array(curr_data[2])
            curr_name = paths[0].split("/")[-1].split("_")[0]
            print (f"model: {i}  dataset {j}/{len(dataset_list)}  img: {curr_name}")
            
            # load sample to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            curr_image = inputs[0]
            curr_label = labels[0].cpu().item()
            # get predictions
            output = model(inputs)
            pred_0 = output.cpu()[0][0].item()
            pred_1 = output.cpu()[0][1].item()

            # log preds
            model_ds_img_names.append(curr_name)
            model_ds_preds_0.append(pred_0)
            model_ds_preds_1.append(pred_1)
            model_ds_img_labels.append(labels.cpu().item())
        # Write current model dataset predictions
        curr_results = pd.DataFrame()
        curr_results['img_name'] = model_ds_img_names
        curr_results['true_label'] = model_ds_img_labels
        curr_results['pred_0'] = model_ds_preds_0
        curr_results['pred_1']= model_ds_preds_1
        curr_results.to_csv(f"{output_base_pth}/preds_{backbone}_{training_strategy}_seed{run_seed[i]}_{dataset_names[j]}.csv", index=False)